<a href="https://colab.research.google.com/github/nathalia-acordi/warehouse-location/blob/main/Warehouse_Location.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qq pyomo
!apt-get install -y -qq glpk-utils

In [ ]:
from pyomo.environ import *

I = [0, 1, 2, 3]  # Exemplos de armazéns
J = [0, 1, 2, 3, 4]  # Exemplos de clientes

f = [1000, 1500, 2000, 1200]  # Custos fixos para abrir cada armazém (incluindo o novo armazém)
c = [
    [2, 4, 5, 3, 6],  # Custos de envio de cada armazém para cada cliente (incluindo o novo armazém)
    [3, 2, 3, 2, 4],
    [4, 3, 2, 1, 5],
    [3, 4, 3, 2, 3],  # Custos de envio de cada armazém para cada cliente (incluindo o novo armazém)
]
d = [80, 270, 250, 160, 180]  # Demanda de cada cliente (incluindo o novo cliente)
C = [500, 500, 500, 600]  # Capacidade de cada armazém (incluindo a capacidade do novo armazém)

# Criar o modelo de otimização
model = ConcreteModel()

# Variáveis de decisão
model.y = Var(I, within=Binary)
model.x = Var(I, J, within=Binary)

# Função objetivo
model.obj = Objective(expr=sum(f[i] * model.y[i] for i in I) +
                         sum(c[i][j] * model.x[i, j] for i in I for j in J),
                      sense=minimize)

# Restrições
model.demand_constraint = ConstraintList()
for j in J:
    model.demand_constraint.add(sum(model.x[i, j] for i in I) == 1)

model.capacity_constraint = ConstraintList()
for i in I:
    model.capacity_constraint.add(sum(d[j] * model.x[i, j] for j in J) <= C[i] * model.y[i])

# Resolver o problema
solver = SolverFactory('glpk')
results = solver.solve(model)

# Imprimir resultados
print(results)
print("Status:", results.solver.status)
print("Termination Condition:", results.solver.termination_condition)

print("Custo Total =", model.obj())
for i in I:
    print("Armazém", i, "está", "aberto" if model.y[i].value == 1 else "fechado")
    for j in J:
        if model.x[i, j].value == 1:
            print("  Cliente", j, "é atendido pelo armazém", i)
